In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(5)

In [2]:
def SGD_SVM(x_train, y_train, iteration, learning_rate,c):

    num_features = len(x_train[0])
    weights = np.random.uniform(low=0.0, high=1.0, size=num_features)


    for t in range(iteration):
        idx = np.random.permutation(len(x_train))
        x_train = x_train[idx]
        y_train = y_train[idx]
        
        for i in range(len(x_train)):
            if (y_train[i] * np.dot(x_train[i], weights)) <= 1:
#                 weights = weights + learning_rate * y_train[i] * x_train[i]
                weights = (1-learning_rate)*weights + learning_rate*c*y_train[i]*x_train[i]
            else:
                weights = (1-learning_rate)*weights
        learning_rate = learning_rate/(1+t)

    return weights

In [3]:
def predict(weights, x_test, y_test):
    prediction = np.sign(np.dot(x_test, weights))

    accuracy = np.mean(prediction == y_test)*100
    return accuracy

In [4]:
def eval_predict(weights, x_test):
   
    prediction = np.sign(np.dot(x_test, weights))
    return prediction

In [5]:
def k_fold_cross_validation():
    data_path = 'data/CVtrain/train{}.csv'
    learning_rate_arr = [1, 0.1, 0.01, 0.001, 0.0001]
    loss_tradeoff = [10,1,0.1,0.01,0.001]
    print("=" * 100)
    

    print("Test and train set of different types ")
    std_acc = []
    avg_acc_arr_lr = []

    best_acc = 0
    best_lr = 0
    best_c = 0
    for lr_rate in learning_rate_arr:

        std_lr_valu = 0
        avg_acc_lr = 0
        

        
        print("*"*10,"new lr_rate:",lr_rate)
        
        for c in loss_tradeoff:
            print("*"*10,"new C:",c)
            avg_acc = 0
            for j in range(5):
                l = list(pd.read_csv(data_path.format(t)) for t in range(1,6) if t != j)
                train_dataframe = pd.concat(l).reset_index(drop=True)
                test_dataframe = pd.read_csv(data_path.format(j+1))

                train_x = train_dataframe.iloc[:, :-1].to_numpy()
                barr = [[float(1)]] * train_dataframe.shape[0]
                train_x = np.append(train_x,barr,axis=1)
                train_y = train_dataframe.iloc[:, -1].to_numpy()
                train_y[train_y == 0] = -1.0

                test_x = test_dataframe.iloc[:, :-1].to_numpy()
                barr = [[float(1)]] * test_dataframe.shape[0]
                test_x = np.append(test_x,barr,axis=1)
                test_y = test_dataframe.iloc[:, -1].to_numpy()
                test_y[test_y == 0] = -1



                train_x = train_x.astype(float)
                train_y = train_y.astype(float)
                test_x = test_x.astype(float)
                test_y = test_y.astype(float)

                # SGD_SVM
                lr_wts = SGD_SVM(train_x, train_y, 5, lr_rate,c)
                prediction = predict(lr_wts, test_x, test_y)
                print("lr: {}, c value: {}, accuracy: {}".format(lr_rate, c, prediction))
                avg_acc +=prediction

            print("avg_acc: ", avg_acc/5)

            if avg_acc/5 > best_acc:
                best_lr = lr_rate
                best_c = c
                best_acc = avg_acc/5
    
     
        std_acc.append(std_lr_valu)
    print("best_lr: {}, best_c: {}, best_acc: {}".format(best_lr, best_c, best_acc))

    # print(std_acc)
    std_max_acc = max(std_acc)
    idx = std_acc.index(std_max_acc)


    print("Best hyper parameter over all cv folds for SGD Version of SVM alg are\n best learning rate: {}\n best loss tradeoff or c: {}\n and best Average accuracy: {}".format(best_lr,best_c, best_acc))
    return best_lr,best_c


In [6]:
def SGD_SVM_Call(train_x, train_y, test_x, test_y):
#     print("*"*100)
    print("SGD Version of SVM Algorithm")
    
    max_acc = 0
    max_predictions = 0
    best_lr = 0
    best_c = 0
    for learning_rate in [1.0,0.1,0.01, 0.001, 0.0001]:
        
        for c in [10,1,0.1,0.01,0.001]:
            
            learned_weights_standard = SGD_SVM(train_x, train_y, 20, learning_rate,c)
            
            train_accuracy = predict(learned_weights_standard, train_x, train_y)
#             print("train_accuracy: ",train_accuracy)
            test_accuracy = predict(learned_weights_standard,  test_x, test_y)
            
            if max_acc < test_accuracy:
                max_acc =  test_accuracy
                best_lr = learning_rate
                best_c = c
#         print("Test Accuracy of SGD Version of SVM with learning rate: {} and c as: {}, is: {}".format(learning_rate, c, test_accuracy))
#         print("Test Accuracy of SGD Version of SVM with learning rate: {} and c as: {}, is: {}".format(learning_rate, c, test_accuracy))
#                 best_lr, best_c = k_fold_cross_validation()
#         print("Test Accuracy of SGD Version of SVM with learning rate: {} and c as: {}, is: {}".format(learning_rate, c, test_accuracy))
#             print("Test Accuracy of SGD Version of SVM with learning rate: {} and c as: {}, is: {}".format(learning_rate, c, test_accuracy))

    return best_lr, best_c
 


In [7]:
if __name__ == "__main__":
    
#     data_path_train = 'data/roberta.train.csv'
#     data_path_test = 'data/roberta.test.csv'
#     data_path_eval = 'data/roberta.eval.anon.csv'

    data_path_train = 'data/spacy.train.csv'
    data_path_test = 'data/spacy.test.csv'
    data_path_eval = 'data/spacy.eval.anon.csv'
    
    train_dataframe = pd.read_csv(data_path_train)
    test_dataframe = pd.read_csv(data_path_test)
    eval_dataframe = pd.read_csv(data_path_eval)

    train_dataframe = train_dataframe.astype(float)
    test_dataframe = test_dataframe.astype(float)
    eval_dataframe = eval_dataframe.astype(float)

    train_x = train_dataframe.iloc[:,:-1].to_numpy()
    barr = [[float(1)]] * train_dataframe.shape[0]
    train_x = np.append(train_x,barr,axis=1)

    
    train_y = train_dataframe.iloc[:, -1].to_numpy()
    train_y[train_y == 0.0] = -1.0
    

    
    test_x = test_dataframe.iloc[:,:-1].values
    barr = [[float(1)]] * test_dataframe.shape[0]
    test_x = np.append(test_x,barr,axis=1)
    
    test_y = test_dataframe.iloc[:, -1].values
    test_y[test_y == 0.0] = -1.0
    
    
    eval_x = eval_dataframe.iloc[:,:-1].to_numpy()
    barr = [[float(1)]] * eval_dataframe.shape[0]
    eval_x = np.append(eval_x,barr,axis=1)

    
    eval_y = eval_dataframe.iloc[:, -1].to_numpy()
    eval_y[eval_y == 0.0] = -1.0
    
    print(train_y)
    print(test_y)
    print(eval_y)
    
    
    



    
    print("*"*100)
#     print("SGD Version of SVM Algorithm")
#     best_lr, best_c = k_fold_cross_validation()
    best_lr, best_c = SGD_SVM_Call(train_x, train_y, test_x, test_y)
    print("Best Learning rate is: {} and best c: {}".format(best_lr, best_c))
    
    learned_weights = SGD_SVM(train_x, train_y, 20, best_lr, best_c)
    train_accuracy = predict(learned_weights, test_x, test_y)
    
    print("Best Test Accuracy of SGD SVM is: {}".format(train_accuracy))
    
    eval_acc = predict(learned_weights, eval_x, eval_y)
    print(eval_acc)

    eval_predictions = eval_predict(learned_weights, eval_x)
    print(eval_predictions)
    
    y_pred = pd.DataFrame(eval_predictions, columns = ["label"], dtype = int)
    y_pred[y_pred == -1] = 0
    y_pred
    
    y_pred.to_csv("data/evals_data_out/evals_SVM_final.csv")
    y_pred.value_counts()
    





[ 1.  1. -1. ...  1. -1. -1.]
[ 1.  1.  1. ...  1. -1.  1.]
[-1. -1. -1. ... -1. -1. -1.]
****************************************************************************************************
SGD Version of SVM Algorithm
Best Learning rate is: 0.001 and best c: 10
Best Test Accuracy of SGD SVM is: 71.8235681369322
49.868421052631575
[ 1. -1. -1. ... -1. -1. -1.]


y_pred = pd.DataFrame(eval_predictions, columns = ["label"], dtype = int)
y_pred[y_pred == -1] = 0
y_pred.to_csv("data/evals_SGD_SVM_final.csv")
y_pred.value_counts()

In [8]:
y_pred.value_counts()

label
1        762
0        758
dtype: int64